In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import h5py


dataDir = '/global/cfs/projectdirs/covid19/sys_uncertainty/test/'
runs = [64, 128, 256, 512, 1024]
age_bins = ['0-4', '5-18', '19-29', '30-64', '65+']

means = {}
stds = {}
for idx, run in enumerate(runs):
    fname=dataDir+'short_'+str(run)+'.h5'
    print("Opening %s"%fname)
    with h5py.File(fname, 'r') as f:
        fields = list(f.keys())
        for field in fields:
            if field not in means.keys():
                means[field] = []
                stds[field] = []
            means[field].append(np.mean(f[field][:,-1,:], axis=0))
            stds[field].append(np.std(f[field][:,-1,:], axis=0))

           
figsz=14
filter1 ='Total individuals by age'
filter2 ='Total unvaccinated individuals by age'
fieldnames = list(filter(lambda x: x!=filter1  and x!=filter2, 
                    means.keys()))
plt.figure(figsize=(figsz,figsz))
idx = 0
for ix in range(3):
    for iy in range(3):
        plt.subplot(3,3,idx+1)
        key = fieldnames[idx]
        plt.title(key)    
        for i in range(len(runs)):
            colors = ['bo', 'ro', 'go', 'mo', 'co']
            if len(means[key][i])==5:
                legend = True
                for j in range(5):
                    plt.errorbar(runs[i], means[key][i][j], yerr=stds[key][i][j], 
                                 fmt=colors[j], label=age_bins[j] if i==0 else None)
                
            else:
                legend = False
                plt.errorbar(runs[i], means[key][i], yerr=stds[key][i], fmt='bo')
        plt.xlabel('Number of runs')
        plt.xscale('log')
        if legend: plt.legend()
        plt.xticks(runs, [str(run) for run in runs])
        plt.gca().get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
        plt.gca().get_xaxis().set_minor_formatter(matplotlib.ticker.NullFormatter())
        if 'rate' not in key:
            plt.yscale('log')
        idx+=1
        
plt.tight_layout(pad=2.0)
plt.show()

In [ ]:

for k,v in means.items():
    if len(v[0]) == 1:
        sigma_m = np.std(v)
        sigma_std = np.std(stds[k])
        print(k, sigma_m/np.mean(v), sigma_std/np.mean(stds[k]))
    else:
        print("%s:"%k)
        for i in range(5):
            sigma_m = np.std([val[i] for val in v])
            norm_m = np.mean([val[i] for val in v])
            sigma_std = np.std([val[i] for val in stds[k]])
            norm_std = np.mean([val[i] for val in stds[k]])
            print('    '+age_bins[i]+':', sigma_m/norm_m, sigma_std/norm_std)